In [1]:
# 2 weeks gap between 2019-04-06 to 2019-04-20
import pandas as pd
import datetime
import os
import numpy as np
import hashlib
import gc
os.getcwd()

'/home/jian/celery/Google_LiveRamp'

In [2]:
def recursive_file_gen(my_root_dir):
    for root, dirs, files in os.walk(my_root_dir):
        for file in files:
            yield os.path.join(root, file)
            


thismonday=datetime.datetime.now().date()-datetime.timedelta(days=datetime.datetime.now().date().weekday())
thismonday=datetime.date(2019,4,22)
print("thismonday", thismonday)

last_week_end_saturday=thismonday-datetime.timedelta(days=2)

print("last_week_end_saturday: ",last_week_end_saturday)

start_since_last_90_days=last_week_end_saturday-datetime.timedelta(days=13)
print("start_since_last_90_days: ",start_since_last_90_days)

thismonday 2019-04-22
last_week_end_saturday:  2019-04-20
start_since_last_90_days:  2019-04-07


In [3]:
last_week_end_saturday

datetime.date(2019, 4, 20)

In [4]:
writer_pather="/home/jian/celery/Google_LiveRamp/output/"
data_root_2019="/home/jian/BigLots/2019_by_weeks/"

In [5]:
files_daily_list=[x for x in list(recursive_file_gen(data_root_2019)) if "Daily" in x]

item_level_daily_df=pd.DataFrame({"file_path":files_daily_list})
item_level_daily_df['week_end_dt']=item_level_daily_df['file_path'].apply(lambda x: x.split("_weeks/MediaStorm_")[1][:10])
item_level_daily_df=item_level_daily_df[(item_level_daily_df['week_end_dt']>=str(start_since_last_90_days)) & (item_level_daily_df['week_end_dt']<=str(last_week_end_saturday))]
item_level_daily_df=item_level_daily_df.sort_values("week_end_dt")
item_level_daily_df

item_level_daily_df['file_path'].tolist()

['/home/jian/BigLots/2019_by_weeks/MediaStorm_2019-04-13/MediaStormDailySales20190416-112824-908.txt',
 '/home/jian/BigLots/2019_by_weeks/MediaStorm_2019-04-20/MediaStormDailySales20190423-112124-771.txt']

In [6]:
i_counter=0
data_to_upload=pd.DataFrame()

for file_item_level in item_level_daily_df['file_path'].tolist():
    df=pd.read_table(file_item_level,sep="|",dtype=str,usecols=['location_id','customer_id_hashed','transaction_dt','item_transaction_amt'])
    df=df[~pd.isnull(df['customer_id_hashed'])]
    df=df[df['location_id']!="6990"]
    df['item_transaction_amt']=df['item_transaction_amt'].astype(float)
    df=df.groupby(['customer_id_hashed','transaction_dt'])['item_transaction_amt'].sum().to_frame().reset_index()
    df=df.rename(columns={"transaction_dt":"transaction_timestamp","item_transaction_amt":"Conversion_Amount"})
    df['transaction_timestamp']=df['transaction_timestamp'].apply(lambda x: datetime.datetime.strptime(x,"%Y-%m-%d").date())
    data_to_upload=data_to_upload.append(df)
    i_counter+=1
    print(i_counter,datetime.datetime.now())
print(data_to_upload['transaction_timestamp'].min())
print(data_to_upload['transaction_timestamp'].max())


1 2019-04-26 15:32:34.603622
2 2019-04-26 15:34:31.012155
2019-04-07
2019-04-20


In [7]:
Master_2019_weekly=[x for x in list(recursive_file_gen("/home/jian/BigLots/2019_by_weeks/")) if ("aster" in x) & (".txt" in x) ]
Master_2018_weekly=[x for x in list(recursive_file_gen("/home/jian/BigLots/2018_by_weeks/")) if ("aster" in x) & (".txt" in x) ]


weekly_df=pd.DataFrame({"file_path":Master_2019_weekly+Master_2018_weekly})
weekly_df['date']=weekly_df['file_path'].apply(lambda x: x.split("_by_weeks/MediaStorm_")[1][:10])
weekly_df['date']=weekly_df['date'].apply(lambda x: datetime.datetime.strptime(x,"%Y-%m-%d").date())
weekly_df=weekly_df.sort_values("date",ascending=False).reset_index()
del weekly_df['index']
weekly_df.head(5)

# No need for the recent posibble file foloder, but added later

,file_path,date
0,/home/jian/BigLots/2019_by_weeks/MediaStorm_20...,2019-04-20
1,/home/jian/BigLots/2019_by_weeks/MediaStorm_20...,2019-04-13
2,/home/jian/BigLots/2019_by_weeks/MediaStorm_20...,2019-04-06
3,/home/jian/BigLots/2019_by_weeks/MediaStorm_20...,2019-03-30
4,/home/jian/BigLots/2019_by_weeks/MediaStorm_20...,2019-03-23


In [8]:
data_0=pd.read_table("/home/jian/Projects/Big_Lots/Loyal_members/loyalty_register_data/MediaStorm_Lapsed_Reward_Member_Master_from2014-08-26to2017-02-26.zip",
                     dtype=str,sep="|",usecols=['customer_id_hashed','email_address_hash','customer_zip_code']).drop_duplicates()
print(data_0.shape)


(1873409, 3)


In [9]:
data_1=pd.read_csv("/home/jian/Projects/Big_Lots/Loyal_members/loyalty_register_data/MediaStormCustTot-hashed-email.txt",
                     dtype=str,header=None,usecols=[0,1,5])
data_1.columns=['customer_id_hashed','email_address_hash','customer_zip_code']
data_1['customer_id_hashed']=data_1['customer_id_hashed'].apply(lambda x: hashlib.sha256(x.encode('utf-8')).hexdigest())
print(data_1.shape)

(19337597, 3)


In [10]:
data_2 = pd.read_csv('/home/jian/Projects/Big_Lots/Loyal_members/loyalty_register_data/'+'MediaStormCustomerTransactionTotals_2018-01-09_2018-03-31.txt',
                     sep = ',',dtype = str,usecols=['customer_id_hashed','email_address_hash','customer_zip_code'])
print(data_2.shape)
data_3 = pd.read_csv('/home/jian/Projects/Big_Lots/Loyal_members/loyalty_register_data/'+'Existing Reward Member Master as of 2018-06-05.txt',
                     dtype = str,sep = '|',usecols=['customer_id_hashed','email_address_hash','customer_zip_code'])
print(data_3.shape)
data_4 = pd.read_csv('/home/jian/Projects/Big_Lots/Loyal_members/loyalty_register_data/'+'New Reward Member Master as of 2018-06-05.txt',
                     dtype = str,sep = '|',usecols=['customer_id_hashed','email_address_hash','customer_zip_code'])
print(data_4.shape)

(7255226, 3)
(5759051, 3)
(659127, 3)


In [11]:
master_old=data_4.append(data_3).append(data_2).append(data_1).append(data_0).drop_duplicates()
del data_4
del data_3
del data_2
del data_1
del data_0
gc.collect()

49

In [12]:
all_weekly_biweekly_master_file=pd.DataFrame()
for file_path in weekly_df['file_path'].tolist():
    df=pd.read_table(file_path,dtype=str,usecols=['customer_id_hashed','email_address_hash','customer_zip_code'],sep="|")
    all_weekly_biweekly_master_file=all_weekly_biweekly_master_file.append(df)

all_weekly_biweekly_master_file=all_weekly_biweekly_master_file.append(master_old)
print(all_weekly_biweekly_master_file.shape)
print(len(all_weekly_biweekly_master_file['customer_id_hashed'].unique()))

all_weekly_biweekly_master_file=all_weekly_biweekly_master_file.drop_duplicates('customer_id_hashed')
print(all_weekly_biweekly_master_file.shape)

(25262335, 3)
25227572
(25227572, 3)


In [13]:
print(data_to_upload.shape)
print(data_to_upload[['customer_id_hashed','transaction_timestamp']].drop_duplicates().shape)

data_to_upload.head(2)

(2834712, 3)
(2834712, 2)


,customer_id_hashed,transaction_timestamp,Conversion_Amount
0,00000435bfb3bf42e3beb4c9b3942c552d09f0e49e5a75...,2019-04-07,13.04
1,00001ccfc0c1cd025b75bb8463ecb11b85b1e4d632b13e...,2019-04-07,21.38


In [14]:
data_to_upload=pd.merge(data_to_upload,all_weekly_biweekly_master_file,on="customer_id_hashed",how="left").rename(columns={"email_address_hash":"Email_1","customer_zip_code":"Zip_Code"})
data_to_upload.head(2)

total_rows_with_na_str=str(data_to_upload.shape[0])
print("Total_Raws: "+total_rows_with_na_str)
na_emails_rows_count_str=str(data_to_upload[pd.isnull(data_to_upload['Email_1'])].shape[0])
print("Null Email rows excluded: "+na_emails_rows_count_str)
data_to_upload=data_to_upload[~pd.isnull(data_to_upload['Email_1'])]
del data_to_upload['customer_id_hashed']

print("Na_Email_Rows_Pctg: ",int(na_emails_rows_count_str)/int(total_rows_with_na_str))

Total_Raws: 2834712
Null Email rows excluded: 182323
Na_Email_Rows_Pctg:  0.06431799773663074


In [15]:
data_to_upload.head(2)

,transaction_timestamp,Conversion_Amount,Email_1,Zip_Code
0,2019-04-07,13.04,e7b239d884e48cdd957205e0ffb8e793bb7b8a391dc102...,21229
5,2019-04-12,0.00,b76d9f13b2d47726edf73e99c553857699217daee94187...,38301


In [16]:
data_to_upload=data_to_upload[["Email_1","Zip_Code","transaction_timestamp", "Conversion_Amount"]].rename(columns={"Conversion_Amount":"transaction_amount"})
data_to_upload['transaction_amount']=data_to_upload['transaction_amount'].apply(lambda x: np.round(x,2)).astype(str)
data_to_upload['transaction_amount']=data_to_upload['transaction_amount'].apply(lambda x: x.split(".")[0]+"."+x.split(".")[1].ljust(2,"0"))
data_to_upload['transaction_category']="In_Store"

data_to_upload['Zip_Code']="00000"

data_to_upload=data_to_upload[['Zip_Code','Email_1','transaction_category','transaction_timestamp','transaction_amount']]

In [17]:
data_max_date=data_to_upload['transaction_timestamp'].max()
data_max_date

datetime.date(2019, 4, 20)

In [18]:
data_min_date=data_to_upload['transaction_timestamp'].min()
data_min_date

datetime.date(2019, 4, 7)

In [19]:
data_to_upload.shape

(2652389, 5)

In [20]:
data_max_date-data_min_date

datetime.timedelta(13)

In [21]:
len(data_to_upload['transaction_timestamp'].unique())

14

In [22]:
writer_pather

'/home/jian/celery/Google_LiveRamp/output/'

In [23]:
local_path=writer_pather+"BL_LR_GoogleStoreSales_2_weeks_"+str(data_min_date)+"_"+str(data_max_date)+"_JL_"+str(datetime.datetime.now().date())+".txt"
data_to_upload.columns.tolist()==['Zip_Code','Email_1','transaction_category','transaction_timestamp','transaction_amount']

data_to_upload.to_csv(local_path,index=False,sep="|")


In [24]:
data_to_upload['transaction_amount'].astype(float).sum()

121112921.32000002

In [25]:
data_to_upload.head(5)

,Zip_Code,Email_1,transaction_category,transaction_timestamp,transaction_amount
0,00000,e7b239d884e48cdd957205e0ffb8e793bb7b8a391dc102...,In_Store,2019-04-07,13.04
5,00000,b76d9f13b2d47726edf73e99c553857699217daee94187...,In_Store,2019-04-12,0.00
6,00000,7518e767bba85d75e674b111c6edfba4c99e55667dac57...,In_Store,2019-04-10,22.43
7,00000,1a9849562f60f238e3ebd2c89842113e5a1ff6e9bae9ee...,In_Store,2019-04-07,78.26
8,00000,3f029ca17f7d6d7c711c18b63ebd245798a8efd2b386d8...,In_Store,2019-04-12,8.00


In [26]:
local_path=="/home/jian/celery/Google_LiveRamp/output/BL_LR_GoogleStoreSales_2_weeks_2019-04-07_2019-04-20_JL_2019-04-26.txt"

True

In [27]:
# Around
data_to_upload['transaction_amount'].astype(float).sum()/215000000

0.56331591311627915

In [28]:
import paramiko

host = "sftp.liveramp.com" #hard-coded
port = 22
transport = paramiko.Transport((host, port))

password = "Jubaplus2019!" #hard-coded
username = "big-lots-ga-aw" #hard-coded
transport.connect(username = username, password = password)
sftp = paramiko.SFTPClient.from_transport(transport)

In [32]:
# local_path defined above before saving the local txt
remote_path="/uploads/"+os.path.basename(local_path)
sftp.put(local_path,remote_path)
sftp.close()